# 🧹 **Bài học: Data Cleaning**

### 🎯 **Mục tiêu học tập**
- Hiểu vai trò của bước **data cleaning** trong quy trình Data Science.
- Nhận biết các dạng dữ liệu bị lỗi, thiếu hoặc ngoại lệ.
- Thực hành các kỹ thuật xử lý **missing data**, **outlier**, và **chuẩn hóa dữ liệu**.
- Ứng dụng các hàm thống kê mô tả để hiểu rõ đặc trưng cơ bản của dữ liệu.
- Phân tích dữ liệu để làm sáng tỏ các mục tiêu báo cáo

## 🧠 **1. Giới thiệu về Data Cleaning**

Trong quy trình **Data Science**, bước làm sạch dữ liệu chiếm đến **60-80% thời gian** của dự án. Dữ liệu thô (raw data) thường chứa lỗi, thiếu sót hoặc định dạng không thống nhất.

Ví dụ về các vấn đề thường gặp:
- Dữ liệu thiếu (`NaN`, `None`).
- Cột dư thừa hoặc vô nghĩa.
- Dữ liệu sai kiểu (ví dụ: số điện thoại lưu dạng chuỗi có ký tự đặc biệt).
- Ngoại lệ (outlier) gây sai lệch thống kê.

Mục tiêu của bước này là giúp **dữ liệu trở nên đáng tin cậy** trước khi phân tích hoặc huấn luyện mô hình.

Nói đơn giản:

👉 **Data cleaning = xử lý lỗi + chuẩn hoá dữ liệu + làm cho dữ liệu trở nên đáng tin cậy.**

## 🧠 **2. QUY TRÌNH XỬ LÝ LÀM SẠCH DỮ LIỆU**

### **Bước 1. Chuẩn bị dữ liệu thô (data raw)**

In [ ]:
import pandas as pd

# Đọc file dữ liệu Customer Call List
df = pd.read_csv('./data/Customer-Call-List.csv')

# Xem trước dữ liệu
df.head()

### **Bước 2. Khám phá và kiểm tra dữ liệu ban đầu**

Trước khi làm sạch, ta cần hiểu cấu trúc và chất lượng dữ liệu thông qua các hàm mô tả và thống kê cơ bản:

In [ ]:
# Kiểm tra kích thước và thông tin tổng quan
df.info()

Qua thông tin nhận được sẽ có cái nhìn tổng quan ban đầu về dữ liệu hiện có. Sau đó bạn tuần tự thực hiện quy trình phân tích thông tin để tìm ra những điểm bất thường của dữ liệu.
> Lưu ý: Với mỗi trường hợp đặc thù thì thì dữ liệu sẽ có điểm bất thường riêng của nó.

1. Tìm dữ liệu rỗng `null`

In [ ]:
# Kiểm tra số lượng giá trị trống (missing values)
df.isnull().sum()

Ví dụ kiểm tra cột `Do_Not_Contact`  có dữ liệu trống không ?

In [ ]:
print("--- Các dòng có giá trị NaN trong 'Do_Not_Contact' ---")
missing_contacts = df[df['Do_Not_Contact'].isnull()]
print(missing_contacts)

Nếu có thì thống kế xem số lượng bao nhiêu để lên phương án xử lý

In [ ]:
# Để kiểm tra nhanh số lượng:
print(df['Do_Not_Contact'].isnull().sum())

2. Tìm Giá trị Bất thường về Định dạng/Loại (Non-Numeric Data)

In [ ]:
# Để kiểm tra nhanh số lượng:
print(df['Do_Not_Contact'].isna().sum())

3. Tìm Giá trị Ngoại lai (`Outliers`)

In [ ]:
import numpy as np

# 1. Xem xét các giá trị duy nhất (unique values)
print("--- Các giá trị Do_Not_Contact duy nhất ---")
print(df['Do_Not_Contact'].unique())

In [ ]:
# 2. Phân tích thống kê mô tả
print("\n--- Thống kê mô tả của 'Do_Not_Contact' ---")
print(df['Do_Not_Contact'].describe())

Giải thích

1. **count**: `17`
   - Số lượng giá trị không bị thiếu (non-null) trong cột `'Do_Not_Contact'` là 17. Điều này có nghĩa là trong tổng số 17 bản ghi, tất cả đều có thông tin trong cột này.

2. **unique**: `4`
   - Có 4 giá trị duy nhất trong cột `'Do_Not_Contact'`. Điều này cho thấy rằng cột này có 4 loại thông tin khác nhau liên quan đến việc có nên liên hệ hay không.

3. **top**: `No`
   - Giá trị phổ biến nhất (thường xuất hiện nhất) trong cột là `'No'`. Điều này có nghĩa là nhiều người trong dữ liệu không muốn được liên hệ.

4. **freq**: `9`
   - Tần suất của giá trị phổ biến nhất (`'No'`) là 9, tức là có 9 bản ghi trong cột này có giá trị là `'No'`.


---

Do dataset này không có giá trị ngoại lai nên ví sau giả lập tình huống tạo ra có.

In [ ]:
# Tạo thêm cột giả định về thời lượng cuộc gọi (phút)
np.random.seed(42)
df['Call_Duration'] = np.random.normal(10, 3, len(df)).round(2)

# Thêm vài ngoại lệ thủ công
df.loc[0, 'Call_Duration'] = 60
df.loc[1, 'Call_Duration'] = 0.5

# Phát hiện outlier bằng IQR
Q1 = df['Call_Duration'].quantile(0.25)
Q3 = df['Call_Duration'].quantile(0.75)
IQR = Q3 - Q1
outliers = df[(df['Call_Duration'] < Q1 - 1.5*IQR) | (df['Call_Duration'] > Q3 + 1.5*IQR)]
outliers

4. Kiểm tra Giá trị Không hợp lệ về Logic (`Invalid Data`)

Giá trị **Không hợp lệ về Logic** (còn gọi là **dữ liệu không hợp logic** hoặc **Invalid Data** trong ngữ cảnh lỗi dữ liệu) là những giá trị tồn tại trong tập dữ liệu nhưng **không tuân theo các quy tắc hoặc mối quan hệ hợp lý** mà dữ liệu đó được kỳ vọng phải có.

Nó khác với giá trị **thiếu** (như `NULL` hoặc ô trống) hay giá trị **sai định dạng** (như nhập chữ vào ô số).

---

💡 Các dạng phổ biến của Lỗi Logic

Lỗi logic thường xảy ra khi giá trị nằm ngoài phạm vi chấp nhận được hoặc mâu thuẫn với các giá trị khác:

* **Giá trị ngoài phạm vi:**
    * **Ví dụ:** Tuổi của một người là **200** hoặc **-5**. (Về mặt kỹ thuật, 200 là một số hợp lệ, nhưng không hợp lý với tuổi người).
    * **Ví dụ:** Ngày sinh lớn hơn ngày hiện tại.

* **Mâu thuẫn giữa các trường:**
    * **Ví dụ:** Một người được ghi là **Học sinh** nhưng lại có chức danh là **Giám đốc**.
    * **Ví dụ:** Ngày bắt đầu công việc sau ngày kết thúc công việc.

* **Giá trị không phù hợp với kiểu dữ liệu đã được làm sạch:**
    * **Ví dụ:** Trong cột **`Paying Customer`** của file bạn tải lên, giá trị hợp lệ được mong đợi là **'Yes'** hoặc **'No'**. Tuy nhiên, có các giá trị biến thể như **'Y'** và **'N'**; nếu logic hệ thống chỉ chấp nhận đầy đủ 'Yes'/'No', thì 'Y'/'N' được coi là không hợp lệ về logic (hoặc cần phải được chuẩn hóa).
    * **Ví dụ:** Số điện thoại có định dạng không thống nhất, chẳng hạn như dùng dấu gạch ngang (`-`), gạch chéo (`/`), hoặc dấu sổ dọc (`|`) trong cột **`Phone_Number`** của file, nếu hệ thống chỉ chấp nhận một định dạng duy nhất.

---

🛠️ Cách nhận biết trong file của bạn

Dựa trên tệp **`Customer-Call-List.csv`** của bạn, đây là một số ví dụ có thể được coi là giá trị không hợp lệ về logic (hoặc cần chuẩn hóa):

| Cột | Giá trị có vấn đề | Lý do (Logic) |
| :--- | :--- | :--- |
| **`Phone_Number`** | `123/643/9775`, `876|678|3469`, `7066950392` | Giá trị có định dạng khác nhau (dùng `/`, `|`, hoặc không có dấu phân cách) trong khi định dạng chuẩn có thể là `XXX-XXX-XXXX`. |
| **`Paying Customer`** | `Y`, `N` | Các giá trị này là viết tắt, có thể không hợp logic nếu quy tắc hệ thống chỉ chấp nhận `Yes` hoặc `No`. |
| **`Last_Name`** | `/White`, trống (cho Gandalf) | Ký tự `/` trong tên hoặc tên trống có thể không hợp logic trong một số ứng dụng. |
| **`Do_Not_Contact`** | `Y`, trống (cho Gandalf) | Tương tự, nếu logic hệ thống chỉ chấp nhận `Yes` hoặc `No`. |
| **`Not_Useful_Column`** | `TRUE`, `FALSE` | Nếu cột này được cho là không hữu ích, nhưng lại chứa các giá trị boolean, có thể cần được xóa hoặc kiểm tra lại mục đích. |

### 🧩 **Bước 3. Xử lý dữ liệu thiếu (Missing Data)**

Các giá trị bị thiếu có thể ảnh hưởng đến việc thống kê và phân tích. Có 3 hướng xử lý phổ biến:

1. **Xoá dòng hoặc cột chứa nhiều giá trị trống.**  
2. **Thay thế giá trị thiếu bằng giá trị trung bình / trung vị / mode.**  
3. **Dự đoán hoặc nội suy giá trị bị thiếu (interpolation).**

👉 Trong ví dụ này, ta sẽ xoá các cột hoàn toàn trống và điền giá trị mặc định cho các ô trống quan trọng.

### 3.1 Xoá các cột không có dữ liệu

Trong quá trình làm sạch dữ liệu, ta thường cần **loại bỏ** (drop) một số **dòng hoặc cột không cần thiết**

Hàm `df.drop()` được sử dụng để **xóa hàng (rows)** hoặc **cột (columns)** khỏi DataFrame.

In [ ]:
# Sử dụng phương thức drop()
df = df.drop(columns=['Unnamed: 8', 'Unnamed: 9'])

### 3.2 Điền giá trị mặc định cho các cột bị thiếu

Khi làm việc với dữ liệu thực tế, ta thường gặp các ô bị thiếu giá trị (`NaN`, `None`, hoặc trống).
Nếu không xử lý, các giá trị này có thể gây lỗi trong tính toán hoặc làm sai lệch kết quả phân tích.

Hàm **`fillna()`** trong Pandas được dùng để **điền (thay thế)** các giá trị bị thiếu bằng một giá trị khác.



🔹 Cú pháp cơ bản

```python
DataFrame.fillna(value=None, method=None, inplace=False)
```

| Tham số   | Ý nghĩa                                                                                                               |
| --------- | --------------------------------------------------------------------------------------------------------------------- |
| `value`   | Giá trị được dùng để điền vào các ô bị thiếu (có thể là số, chuỗi hoặc từ điển `{column: value}`)                     |
| `method`  | Cách điền dữ liệu tự động, có thể là `'ffill'` (điền theo giá trị trước đó) hoặc `'bfill'` (điền theo giá trị sau đó) |
| `inplace` | Nếu `True`, thay đổi trực tiếp DataFrame gốc mà không cần gán lại biến mới                                            |

---

In [ ]:
# Điền giá trị mặc định cho các cột bị thiếu
df.fillna({
    'Last_Name': 'Unknown',
    'Phone_Number': 'Missing',
    'Do_Not_Contact': 'No Info'
}, inplace=True)

Hoặc Gán trực tiếp (rõ ràng, không warning)

In [ ]:
# Điền giá trị mặc định cho các cột bị thiếu
df['Last_Name'] = df['Last_Name'].fillna('Unknown')
df['Phone_Number'] = df['Phone_Number'].fillna('Missing')
df['Do_Not_Contact'] = df['Do_Not_Contact'].fillna('No Info')

In [ ]:
# Review lại dữ liệu
df.head()

💡 Ghi nhớ

* Dùng `fillna()` khi bạn **không muốn mất dữ liệu** do xoá dòng (`dropna()`).
* Kiểm tra trước khi điền bằng `df.isnull().sum()`.
* Thường dùng để **làm sạch dữ liệu thiếu (missing data)** trong bước *Data Cleaning*.
* Có thể áp dụng theo từng cột, toàn bộ DataFrame, hoặc dựa vào thống kê như mean/median/mode.


### 🧱 **Bước 4. Xử lý dữ liệu không hợp lệ và chuẩn hóa định dạng**

Một số giá trị có thể được nhập sai hoặc không thống nhất, ví dụ:
- Cột `Phone_Number` có các ký tự `/`, `|`, `-`.
- Cột `Paying Customer` có giá trị `Yes`, `No`, `Y`, `N`.
- Cột `Do_Not_Contact` có `Y`, `Yes`, `No`, `N`, `NaN`.

Chúng ta cần chuẩn hóa lại các giá trị này để phân tích dễ dàng hơn.

In [ ]:
# Làm sạch định dạng số điện thoại
df['Phone_Number'] = df['Phone_Number'].str.replace(r'[^0-9]', '', regex=True)
# Kiểm tra lại dữ liệu sau khi chuẩn hóa
df.head()

In [ ]:
# Chuẩn hoá cột Paying Customer
df['Paying Customer'] = df['Paying Customer'].replace({'Y': 'Yes', 'N': 'No'})
# Kiểm tra lại dữ liệu sau khi chuẩn hóa
df.head()

In [ ]:
# Chuẩn hoá cột Do_Not_Contact
df['Do_Not_Contact'] = df['Do_Not_Contact'].replace({'Y': 'Yes', 'N': 'No'})

# Kiểm tra lại dữ liệu sau khi chuẩn hóa
df.head()

### ⚠️ **Bước 5. Xử lý dữ liệu ngoại lệ (Outliers)**

Ngoại lệ (outlier) là các giá trị khác biệt bất thường so với phần còn lại. Ví dụ, nếu ta có cột `Age` hoặc `Income`, giá trị cực lớn hoặc cực nhỏ có thể ảnh hưởng tới trung bình.

👉 Dữ liệu hiện tại không có biến số học, nên ta có thể minh họa bằng cách tạo thêm cột `Call_Duration` để ví dụ xử lý ngoại lệ.

Sử dụng **quy tắc IQR (Interquartile Range)**:

In [ ]:
import numpy as np

# Tạo thêm cột giả định về thời lượng cuộc gọi (phút)
np.random.seed(42)
df['Call_Duration'] = np.random.normal(10, 3, len(df)).round(2)

# Thêm vài ngoại lệ thủ công
df.loc[0, 'Call_Duration'] = 60
df.loc[1, 'Call_Duration'] = 0.5
df # xem trước kết quả

In [ ]:
# Phát hiện outlier bằng IQR
Q1 = df['Call_Duration'].quantile(0.25)
Q3 = df['Call_Duration'].quantile(0.75)
IQR = Q3 - Q1

# Xác định ngưỡng để xác định ngoại lệ
lower_bound = Q1 - 1.5 * IQR
upper_bound = Q3 + 1.5 * IQR

outliers = df[(df['Call_Duration'] < lower_bound) | (df['Call_Duration'] > upper_bound)]
print("Giá trị ngoại lệ:")
outliers

Như vậy phát hiện ra một ngoại lệ có `thời lượng gọi` lớn

#### Các cách thức xử lý ngoại lệ

**Cách 1. Xóa Ngoại lệ**

In [ ]:
df_filtered = df[(df['Call_Duration'] >= lower_bound) & (df['Call_Duration'] <= upper_bound)]

**Cách 2 Thay thế Ngoại lệ**

In [ ]:
# Thay thế bằng giá trị trung vị
df.loc[df['Call_Duration'] < lower_bound, 'Call_Duration'] = df['Call_Duration'].median()
df.loc[df['Call_Duration'] > upper_bound, 'Call_Duration'] = df['Call_Duration'].median()

**Cách 3. Ghi chú Ngoại lệ**

Thay vì loại bỏ hoặc thay thế, bạn có thể muốn ghi chú lại các ngoại lệ để xem xét sau.

In [ ]:
df['Is_Outlier'] = df['Call_Duration'].apply(lambda x: 'Yes' if (x < lower_bound or x > upper_bound) else 'No')

**Cách 4. Winsorizing (Capping)** – ĐƯỢC KHUYẾN CÁO NHẤT cho Call Duration

In [ ]:
# Tính ngưỡng theo IQR
lower_bound = Q1 - 1.5 * IQR
upper_bound = Q3 + 1.5 * IQR

# Capping (rất phổ biến trong báo cáo KPI call center)
df['Call_Duration_Capped'] = df['Call_Duration'].clip(lower=lower_bound, upper=upper_bound)

# Hoặc cap theo percentile (thường dùng hơn IQR vì ổn định hơn)
df['Call_Duration_Winsor'] = df['Call_Duration'].clip(
    lower=df['Call_Duration'].quantile(0.01),
    upper=df['Call_Duration'].quantile(0.99)
)

df

Dưới đây là **tổng hợp đầy đủ và thực tế nhất** các cách xử lý outliers trong pandas mà dân Data Analyst / Data Scientist Việt Nam hay dùng từ dự án nhỏ đến dự án lớn (cập nhật 2025).

| STT | Phương pháp                  | Code pandas ngắn gọn                                                                                         | Khi nào dùng? (thực tế)                                                                                   | Ưu điểm                                                                 | Nhược điểm                                                              |
|-----|------------------------------|--------------------------------------------------------------------------------------------------------------|----------------------------------------------------------------------------------------------------------|-------------------------------------------------------------------------|-------------------------------------------------------------------------|
| 1   | **Winsorizing (Capping)** – HOT NHẤT | ```df
|     |                              | hoặc dùng percentile                                                                                         |                                                                                                          |                                                                         |                                                                         |
|     |                              | ```df['col_win'] = df['col'].clip(df['col'].quantile(0.01), df['col'].quantile(0.99))```                    |                                                                                                          |                                                                         |                                                                         |
| 2   | **Thay bằng giá trị giới hạn IQR** | ```lower = Q1 - 1.5*IQR; upper = Q3 + 1.5*IQR```<br>```df['col'] = df['col'].clip(lower, upper)```          | Dữ liệu tài chính, doanh thu, chi phí                                                                    | Dễ giải thích với sếp                                                   | Có thể cắt quá mạnh nếu dữ liệu lệch nặng                               |
| 3   | **Thay bằng median / mean**   | ```df['col'] = df['col'].where(cond, df['col'].median())```                                                  | Khi outlier chắc chắn là lỗi nhập liệu                                                                   | An toàn, không làm thay đổi trung vị                                    | Làm mất thông tin thực tế của outlier                                   |
| 4   | **Log transformation**       | ```df['col_log'] = np.log1p(df['col'])```                                                                    | Xây mô hình ML, dữ liệu lệch phải rất mạnh (doanh thu, lương…)                                           | Giảm ảnh hưởng outlier cực tốt                                          | Khó giải thích với người không kỹ thuật                                 |
| 5   | **Tạo cột flag + giữ nguyên** (best practice) | ```df['is_outlier'] = (df['col'] < lower) | (df['col'] > upper)```<br>```df['col_clean'] = df['col'].clip(...)``` | Phân tích riêng nhóm “bất thường”, báo cáo chi tiết                                                      | Rất linh hoạt, không mất thông tin                                      | Tăng số cột                                                             |
| 6   | **Loại bỏ hoàn toàn**         | ```df_clean = df[~df['is_outlier']].copy()```                                                                | Chỉ khi chắc 100% là lỗi nhập liệu (ví dụ: tuổi = 200, lương = -5000)                                    | Dữ liệu cực sạch                                                        | Mất thông tin khách hàng quan trọng → nguy hiểm                         |
| 7   | **Percentile-based replace** | ```df.loc[df['col'] > q99, 'col'] = q99```                                                                   | Doanh thu khách hàng VIP, thời lượng cuộc gọi rất dài                                                    | Giữ hình dạng phân phối tốt hơn IQR                                     |                                                                         |
| 8   | **Z-score (ít dùng với dữ liệu kinh doanh)** | ```from scipy import stats```<br>```df['z'] = np.abs(stats.zscore(df['col']))```<br>```outliers = df[df['z'] > 3]``` | Dữ liệu gần chuẩn (normal distribution) như chiều cao, cân nặng…                                        | Khoa học                                                                | Không phù hợp hầu hết dữ liệu kinh doanh (lệch phải)                    |
| 9   | **Robust Scaling (dùng cho ML)** | ```from sklearn.preprocessing import RobustScaler```<br>```df[['col']] = RobustScaler().fit_transform(df[['col']])``` | Chuẩn bị dữ liệu cho mô hình ML                                                                          | Giảm ảnh hưởng outlier cực mạnh                                         | Giá trị không còn ý nghĩa nghiệp vụ                                     |
| 10  | **Quantile Transformer (pro)** | ```from sklearn.preprocessing import QuantileTransformer```<br>```qt = QuantileTransformer(output_distribution='normal')``` | Khi cần đưa dữ liệu về chuẩn hoặc uniform để train model                                                 | Outlier gần như bị triệt tiêu                                           | Khó giải thích                                                          |

### Top 3 cách ĐƯỢC DÙNG NHIỀU NHẤT ở công ty Việt Nam

```python
# 1. Winsorize 1%-99% – chuẩn báo cáo sếp
df['Amount_clean'] = df['Amount'].clip(
    df['Amount'].quantile(0.01),
    df['Amount'].quantile(0.99)
)

# 2. Tạo flag + clean song song – chuẩn dashboard Power BI
df['Revenue_flag'] = np.where(df['Revenue'] > df['Revenue'].quantile(0.99), 
                              'Top 1% Revenue', 'Normal')
df['Revenue_clean'] = df['Revenue'].clip(upper=df['Revenue'].quantile(0.99))

# 3. IQR capping – nhanh, dễ hiểu
Q1 = df['Call_Duration'].quantile(0.25)
Q3 = df['Call_Duration'].quantile(0.75)
IQR = Q3 - Q1
df['Call_Duration_clean'] = df['Call_Duration'].clip(Q1 - 1.5*IQR, Q3 + 1.5*IQR)
```

### Quy tắc vàng chọn cách xử lý
| Loại dữ liệu                  | Cách xử lý nên dùng                          |
|-------------------------------|----------------------------------------------|
| Thời lượng cuộc gọi           | Winsorize 1%-99% + flag                      |
| Doanh thu, đơn hàng           | Winsorize 1%-99% hoặc thay bằng 99th percentile |
| Tuổi, số lượng sản phẩm       | IQR hoặc loại bỏ nếu > 150 tuổi              |
| Lương nhân viên               | Winsorize 0.5%-99.5%                         |
| Xây mô hình ML                | RobustScaler / QuantileTransformer           |
| Báo cáo chính thức cho sếp    | Luôn giữ bản sạch + bản gốc + cột flag       |

### 📊 **Bước 6. Phân tích thống kê mô tả (nếu cần)**

Các hàm thống kê giúp chúng ta hiểu rõ hơn về đặc điểm dữ liệu sau khi làm sạch:
- `df.describe()` : Thống kê cơ bản cho cột số.
- `df['col'].value_counts()` : Đếm tần suất xuất hiện.
- `df.corr()` : Phân tích tương quan (nếu có biến số học).

In [ ]:
# Thống kê mô tả cơ bản
df.describe()

In [ ]:
# Đếm tần suất Paying Customer
df['Paying Customer'].value_counts()

Sau khi làm sạch dữ liệu qua 6 bước trên. Chúng ta sẵn sàng đưa nó vào để phân tích những bài toán mục tiêu theo yêu cầu

## **3. Phân tích dữ liệu**

Qua phần dữ liệu về quản lý cuộc gọi chăm sóc khách hàng trên thì có thể xảy ra một số kịch bản phân tích dữ liệu như sau:

> Lưu ý: Phần `Trực quan hoá nội dung` là định hướng cho phần lý thuyết bài tiếp theo.

#### **3.7.1 Phân phối Khách hàng Trả Tiền so với Khách hàng Không Trả Tiền**

* **Mục tiêu:** Hiểu tỷ lệ cơ bản của khách hàng hiện tại (**Paying Customer** = 'Yes') so với khách hàng không trả tiền (**Paying Customer** = 'No').
* **Trực quan hóa:** **Biểu đồ tròn (Pie Chart)** hoặc **Biểu đồ thanh (Bar Chart)**.
* **Giá trị:** Giúp xác định quy mô của cơ sở khách hàng hiện tại và tiềm năng.


In [ ]:
#Code ở đây
counts = df['Paying Customer'].value_counts()
print(counts)
# Lấy dữ liệu này đổ lên biểu đồ

#### **3.7.2. Số lượng Khách hàng cần Tránh Liên hệ**

* **Mục tiêu:** Xác định số lượng khách hàng được đánh dấu là không được liên hệ (**Do_Not_Contact** = 'Yes').
* **Trực quan hóa:** **Biểu đồ thanh (Bar Chart)** đơn giản hoặc **Biểu đồ tròn (Pie Chart)** cho tỷ lệ "Nên Liên hệ" và "Không Nên Liên hệ".
* **Giá trị:** Cực kỳ quan trọng để lọc dữ liệu trước khi gọi điện, tránh lãng phí thời gian và làm phiền khách hàng.


In [ ]:
# Giả sử cột đã sạch: chỉ có 'Yes' và 'No'
avoid_count = (df['Do_Not_Contact'] == 'Yes').sum()
total_count = len(df)

print(f"Số khách hàng cần TRÁNH liên hệ: {avoid_count} / {total_count} khách")
print(f"Tỷ lệ phải tránh: {avoid_count / total_count:.1%}")

#### **3.7.3. Phân tích Tình trạng Trả tiền kết hợp với Tình trạng Không Liên hệ**

* **Mục tiêu:** Xem xét mối quan hệ giữa việc khách hàng có trả tiền hay không và việc họ có được đánh dấu là không nên liên hệ hay không.
* **Trực quan hóa:** **Biểu đồ thanh được nhóm (Grouped Bar Chart)**. Trục X là **Paying Customer**, các thanh được nhóm theo **Do_Not_Contact**.
* **Giá trị:** Có thể cho thấy liệu có xu hướng khách hàng trả tiền hoặc không trả tiền yêu cầu không liên hệ hay không.


In [ ]:
# Tạo bảng chéo (crosstab) 
cross_table = pd.crosstab(
    df['Paying Customer'],
    df['Do_Not_Contact'],
    margins=True,
    margins_name='Total'
)

# Thêm % theo hàng (rất quan trọng)
cross_table_pct = pd.crosstab(
    df['Paying Customer'], 
    df['Do_Not_Contact'], 
    normalize='index'
).round(4)*100

print("BẢNG PHÂN TÍCH: PAYING CUSTOMER KẾT HỢP DO NOT CONTACT")
print(cross_table)
print("\nTỷ lệ % theo nhóm Paying:")
print(cross_table_pct.map('{:.1f}%'.format))

## 🧪 **4. Thực hành: Làm sạch và Phân tích mô tả dữ liệu**